## 1. Import Libraries

In [15]:
import os

import boto3

import pickle

import warnings

import numpy as np

import pandas as pd

import xgboost as xgb

import sklearn
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
	OneHotEncoder,
	OrdinalEncoder,
	StandardScaler,
	MinMaxScaler,
	PowerTransformer,
	FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

## 2. Display Settings

In [16]:
pd.set_option("display.max_columns", None)

In [17]:
sklearn.set_config(transform_output="pandas")

In [18]:
warnings.filterwarnings("ignore")

## 3. Read Datasets

In [19]:
train = pd.read_csv(r"C:\Sahil\portfolio\Flight-Fare-Forecasting-with-AWS-SageMaker\data\train.csv")
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-05-15,Delhi,Cochin,04:55:00,21:00:00,965,1.0,No Info,6979
1,Jet Airways,2019-03-27,Delhi,Cochin,09:00:00,18:50:00,590,1.0,In-flight meal not included,5963
2,Indigo,2019-06-12,Delhi,Cochin,16:00:00,01:30:00,570,1.0,No Info,6316
3,Indigo,2019-04-12,Banglore,Delhi,13:00:00,15:50:00,170,0.0,No Info,4823
4,Indigo,2019-05-18,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6704
...,...,...,...,...,...,...,...,...,...,...
635,Air India,2019-05-24,Kolkata,Banglore,07:00:00,20:25:00,805,1.0,No Info,10413
636,Indigo,2019-06-12,Chennai,Kolkata,11:35:00,14:00:00,145,0.0,No Info,3858
637,Jet Airways,2019-03-27,Delhi,Cochin,09:00:00,18:50:00,590,1.0,No Info,12032
638,Jet Airways,2019-03-24,Kolkata,Banglore,13:55:00,04:45:00,890,1.0,In-flight meal not included,10661


In [20]:
val = pd.read_csv(r"C:\Sahil\portfolio\Flight-Fare-Forecasting-with-AWS-SageMaker\data\val.csv")
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-06-09,Delhi,Cochin,11:35:00,01:30:00,835,3.0,No Info,15419
1,Jet Airways,2019-06-09,Kolkata,Banglore,06:30:00,04:40:00,1330,1.0,No Info,14178
2,Indigo,2019-05-15,Delhi,Cochin,21:05:00,00:20:00,195,0.0,No Info,6834
3,Jet Airways,2019-04-15,Banglore,Delhi,17:45:00,20:45:00,180,0.0,No Info,7229
4,Jet Airways,2019-06-06,Delhi,Cochin,09:40:00,04:25:00,1125,2.0,No Info,13014
...,...,...,...,...,...,...,...,...,...,...
155,Jet Airways,2019-05-21,Kolkata,Banglore,16:30:00,23:35:00,425,1.0,No Info,14781
156,Multiple Carriers,2019-03-21,Delhi,Cochin,04:45:00,18:50:00,845,1.0,No Info,11098
157,Multiple Carriers,2019-05-18,Delhi,Cochin,10:35:00,19:15:00,520,1.0,No Info,8309
158,Jet Airways,2019-05-06,Kolkata,Banglore,06:30:00,10:05:00,1655,1.0,No Info,13941


In [21]:
test = pd.read_csv(r"C:\Sahil\portfolio\Flight-Fare-Forecasting-with-AWS-SageMaker\data\test.csv")
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Goair,2019-05-18,Kolkata,Banglore,23:30:00,07:45:00,495,1.0,No Info,7335
1,Vistara,2019-06-03,Chennai,Kolkata,07:05:00,09:20:00,135,0.0,No Info,3687
2,Jet Airways,2019-06-01,Delhi,Cochin,05:30:00,04:25:00,1375,2.0,In-flight meal not included,10368
3,Indigo,2019-03-01,Delhi,Cochin,04:55:00,21:20:00,985,2.0,No Info,10278
4,Jet Airways,2019-05-21,Kolkata,Banglore,16:30:00,22:35:00,365,1.0,No Info,14781
...,...,...,...,...,...,...,...,...,...,...
195,Jet Airways,2019-06-27,Delhi,Cochin,22:50:00,04:25:00,335,1.0,No Info,16079
196,Multiple Carriers,2019-03-21,Delhi,Cochin,08:25:00,18:50:00,625,1.0,No Info,13069
197,Jet Airways,2019-03-27,Delhi,Cochin,19:10:00,04:25:00,1995,2.0,No Info,10929
198,Jet Airways,2019-03-21,Delhi,Cochin,07:05:00,04:25:00,1280,1.0,No Info,12767


## 4. Preprocessing Operations

In [22]:
# airline
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scaler", MinMaxScaler())
])

# source & destination
location_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", MeanEncoder()),
    ("scaler", PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
    return (
        X
        .assign(**{
            f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

location_transformer = FeatureUnion(transformer_list=[
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func=is_north))
])

# dep_time & arrival_time
time_pipe1 = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
    ("scaler", MinMaxScaler())
])

def part_of_day(X, morning=4, noon=12, eve=16, night=20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col: pd.to_datetime(X.loc[:, col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_temp.loc[:, col].between(morning, noon, inclusive="left"),
                 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
                 X_temp.loc[:, col].between(eve, night, inclusive="left")],
                ["morning", "afternoon", "evening"],
                default="night"
            )
            for col in columns
        })
        .drop(columns=columns)
    )

time_pipe2 = Pipeline(steps=[
    ("part", FunctionTransformer(func=part_of_day)),
    ("encoder", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list=[
    ("part1", time_pipe1),
    ("part2", time_pipe2)
])

# duration
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma


    def fit(self, X, y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include="number").columns.to_list()

        self.reference_values_ = {
            col: (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1, 1)
            )
            for col in self.variables
        }

        return self


    def transform(self, X):
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
                columns=columns
            )
            objects.append(obj)
        return pd.concat(objects, axis=1)
    

def duration_category(X, short=180, med=400):
    return (
        X
        .assign(duration_cat=np.select([X.duration.lt(short),
                                    X.duration.between(short, med, inclusive="left")],
                                    ["short", "medium"],
                                    default="long"))
        .drop(columns="duration")
    )

def is_over(X, value=1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}": X.duration.ge(value).astype(int)
        })
        .drop(columns="duration")
    )

duration_pipe1 = Pipeline(steps=[
    ("rbf", RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
    ("cat", FunctionTransformer(func=duration_category)),
    ("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func=is_over)),
    ("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
    ("imputer", SimpleImputer(strategy="median")),
    ("union", duration_union)
])

# total_stops
def is_direct(X):
    return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("", FunctionTransformer(func=is_direct))
])

# additional_info
info_pipe1 = Pipeline(steps=[
    ("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
    return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
("part1", info_pipe1),
("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
("union", info_union)
])

# column transformer
column_transformer = ColumnTransformer(transformers=[
("air", air_transformer, ["airline"]),
("doj", doj_transformer, ["date_of_journey"]),
("location", location_transformer, ["source", 'destination']),
("time", time_transformer, ["dep_time", "arrival_time"]),
("dur", duration_transformer, ["duration"]),
("stops", total_stops_transformer, ["total_stops"]),
("info", info_transformer, ["additional_info"])
], remainder="passthrough")

# feature selector
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
estimator=estimator,
scoring="r2",
threshold=0.1
) 

# preprocessor
preprocessor = Pipeline(steps=[
("ct", column_transformer),
("selector", selector)
])

In [23]:
preprocessor.fit(
    train.drop(columns="price"),
    train.price.copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetime...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x00000192C4484900>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [24]:
preprocessor.transform(train.drop(columns="price"))

,air__airline_Indigo,air__airline_Jet Airways,doj__date_of_journey_month,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,dur__duration_rbf_25,dur__duration_cat,dur__duration,stops__total_stops,stops__is_direct_flight
0,1.0,0.0,0.666667,0.647059,0.635593,1.080268,1.075131,-0.367289,2.0,0.658505,1.0,0
1,0.0,1.0,0.000000,0.235294,0.220339,1.080268,1.075131,-0.367289,2.0,-0.081765,1.0,0
2,1.0,0.0,1.000000,0.882353,0.872881,1.080268,1.075131,-0.367289,2.0,-0.121246,1.0,0
3,1.0,0.0,0.333333,0.352941,0.355932,-0.670759,-1.822191,3.096271,0.0,-0.910867,0.0,1
4,1.0,0.0,0.666667,0.647059,0.661017,-0.278634,-0.301131,-0.367289,1.0,-0.604889,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
635,0.0,0.0,0.666667,0.705882,0.711864,-0.278634,-0.301131,-0.367289,2.0,0.342656,1.0,0
636,1.0,0.0,1.000000,0.882353,0.872881,-1.830448,-0.590573,-0.367289,0.0,-0.960219,0.0,1
637,0.0,1.0,0.000000,0.235294,0.220339,1.080268,1.075131,-0.367289,2.0,-0.081765,1.0,0
638,0.0,1.0,0.000000,0.176471,0.194915,-0.278634,-0.301131,-0.367289,2.0,0.510451,1.0,0


## 4. Preprocess Data and Upload to Bucket

In [25]:
BUCKET_NAME = "sagemaker-flights-bucket"

DATA_PREFIX = "data"

In [26]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [27]:
def export_data(data, name, pre):
    # split data into X and y subsets
    X = data.drop(columns="price")
    y = data.price.copy()
    
    # transformation
    X_pre = pre.transform(X)
    
    # exporting
    file_name = get_file_name(name)
    (
        y
        .to_frame()
        .join(X_pre)
        .to_csv(file_name, index=False, header=False)
    )

In [28]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )

In [29]:
def export_and_upload_bucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [31]:
# export_and_upload_bucket(train, "train", preprocessor)

In [33]:
# export_and_upload_bucket(val, "val", preprocessor)

In [34]:
# export_and_upload_bucket(test, "test", preprocessor)

## 5. Model and Hyperparameter Tuning Set-up

In [36]:
# session = sagemaker.Session()
# region_name = session.boto_region_name

In [ ]:
# output_path = f"s3://{BUCKET_NAME}/model/output"

In [38]:
# model = Estimator(
#     image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
#     role=sagemaker.get_execution_role(),
#     instance_count=1,
#     instance_type="ml.m4.xlarge",
#     volume_size=5,
#     output_path=output_path,
#     use_spot_instances=True,
#     max_run=300,
#     max_wait=600,
#     sagemaker_session=session
# )

In [40]:
# model.set_hyperparameters(
#     objective="reg:linear",
#     num_round=10,
#     eta=0.1,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     alpha=0.1
# )

In [41]:
# hyperparameter_ranges = {
#     "eta": ContinuousParameter(0.05, 0.2),
#     "alpha": ContinuousParameter(0, 1),
#     "max_depth": IntegerParameter(3, 5)
# }

In [43]:
# tuner = HyperparameterTuner(
#     estimator=model,
#     objective_metric_name="validation:rmse",
#     hyperparameter_ranges=hyperparameter_ranges,
#     strategy="Bayesian",
#     objective_type="Minimize"
# )

## 6. Data Channels

In [ ]:
# def get_data_channel(name):
#     bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
#     return TrainingInput(bucket_path, content_type="csv")

In [ ]:
# train_data_channel = get_data_channel("train")
# train_data_channel

In [ ]:
# val_data_channel = get_data_channel("val")

In [ ]:
# data_channels = {
#     "train": train_data_channel,
#     "validation": val_data_channel
# }

## 7. Train and Tune the Model

In [44]:
# tuner.fit(data_channels)

NameError: name 'tuner' is not defined

## 8. Model Evaluation

In [ ]:
# with open("xgboost-model", "rb") as f:
#     best_model = pickle.load(f)
    
# best_model

In [ ]:
# def evaluate_model(name):
#     file_name = get_file_name(name)
#     data = pd.read_csv(file_name)
    
#     X = xgb.DMatrix(data.iloc[:, 1:])
#     y = data.iloc[:, 0].copy()
    
#     pred = best_model.predict(X)
    
#     return r2_score(y, pred)

In [ ]:
# evaluate_model("train")

-0.5000379618028745

In [ ]:
# evaluate_model("val")

-0.4516233294836658

In [ ]:
# evaluate_model("test")

-0.43416313174551946